## Run Configuration

In [ ]:
import sys
import pathlib

# In Jupyter, use cwd() instead of __file__
ROOT = pathlib.Path.cwd().parent.resolve()
sys.path.append(str(ROOT))

In [ ]:
import time

config = {
    "datasets": ["TramCanGio"],
    "target": "EC[g/l]",
    "resample_freq": "h",
    "resample_agg": "max",
    "prediction_length": 24,
    "eval_metric": "MSE",
    "presets": "fast_training",
    "time_limit": 60,
}


config["name"] = f"autogluon_{config['resample_freq']}"


## Data loading

In [ ]:
from src.scripts.dataloader.factory import load_datasets

df = load_datasets(config["datasets"], resample_freq=config["resample_freq"], resample_agg=config["resample_agg"])
df.head()

In [ ]:
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame

long_data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="station",
    timestamp_column="ds"
)

# train_df, test_df = long_data.train_test_split(config["prediction_length"] * 3)
train_df = long_data.slice_by_timestep(0, -config["prediction_length"]*3)
test_df = long_data.slice_by_timestep(-config["prediction_length"]*3, None)

train_df.tail(), test_df.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

for station in train_df.item_ids:

    train_ts = train_df.loc[station]
    test_ts  = long_data.loc[station]

    # Single plot
    fig, ax = plt.subplots(figsize=(12, 4))

    # Plot test
    ax.plot(test_ts.index, test_ts.values, label="Test")

    # Highlight forecast horizon
    ax.axvspan(
        train_ts.index[-1],       # boundary between past and future
        test_ts.index[-1],        # end of test data
        color="orange",
        alpha=0.3,
        label="Forecast horizon",
    )

    ax.set_title(f"{station} – Test (Historical + Future)")
    ax.grid(True)
    ax.legend()
    plt.tight_layout()
    plt.show()


## Training

In [ ]:
import wandb

wandb.login(relogin=True)

In [ ]:
wandb.init(project="BASWAP", config=config)

In [ ]:
from autogluon.timeseries import TimeSeriesPredictor

predictor = TimeSeriesPredictor(
    prediction_length=config["prediction_length"],
    path=f"{ROOT}/models/autogluonTS/{config["name"]}",
    target=config["target"],
    eval_metric=config["eval_metric"],
    freq=config["resample_freq"],
)

predictor.fit(
    train_df,
    presets=config["presets"],
    time_limit=config["time_limit"],
)

In [ ]:
import json

results_dict = predictor.fit_summary()

rows = []
for m in predictor.model_names():
    rows.append({
        "model": m,
        "model_type": results_dict["model_types"].get(m),
        "performance": results_dict["model_performance"].get(m),  
        "is_best": (m == results_dict.get("model_best")),
        "fit_time": results_dict["model_fit_times"].get(m),
        "pred_time": results_dict["model_pred_times"].get(m),
        "hyperparams": json.dumps(results_dict["model_hyperparams"].get(m, {})),
    })

summary_df = pd.DataFrame(rows)
summary_table = wandb.Table(data=summary_df, columns=summary_df.columns.tolist())
wandb.log({"train_summary": summary_table})

In [ ]:
# Get leaderboard
lb = predictor.leaderboard(
    test_df,
    extra_metrics=["MAE", "MSE", "RMSE"],
    silent=True
)

# Convert to W&B table
lb_table = wandb.Table(dataframe=lb)

# Log
wandb.log({"test_leaderboard": lb_table})

In [ ]:
# rows = []
# for model in predictor.model_names():
#     result = predictor.evaluate(test_df, metrics=["MAE", "MSE", "RMSE"], model=model)
#     row = {"model": model, **result}
#     rows.append(row)

# test_result = pd.DataFrame(rows)

# table = wandb.Table(data=test_result, columns=["model", "MAE", "MSE", "RMSE"])
# wandb.log({"test_results": table})

In [ ]:
import numpy as np
import pandas as pd

H = config["prediction_length"]
input_window = 2 * H

rows = []

model_names = predictor.model_names()

for model in model_names:

    abs_errors_per_lead = [[] for _ in range(H)]
    sq_errors_per_lead  = [[] for _ in range(H)]

    for item in test_df.item_ids:

        series_true = test_df.loc[item][config["target"]]

        t_arr = np.asarray(series_true)
        N = len(t_arr)
        if N < input_window + H:
            continue

        num_rolls = N - (input_window + H) + 1

        for roll in range(num_rolls):

            input_start = 0
            input_end   = roll + input_window
            future_start = input_end
            future_end   = input_end + H

            # --- FIX: AutoGluon REQUIRES a TimeSeriesDataFrame, not a Series ---
            input_slice = test_df.loc[[item]].iloc[input_start:input_end]

            # Predict only this item
            pred_df = predictor.predict(input_slice, model=model)

            # AG output: MultiIndex → (item, timestamp) → 'mean'
            pred_series = pred_df.loc[item]["mean"].values

            true_future = t_arr[future_start:future_end]

            min_len = min(len(true_future), len(pred_series))
            if min_len == 0:
                continue

            for j in range(min_len):
                t = true_future[j]
                p = pred_series[j]
                if np.isnan(t) or np.isnan(p):
                    continue

                err = p - t
                abs_errors_per_lead[j].append(abs(err))
                sq_errors_per_lead[j].append(err ** 2)

    # --- Compute metrics for each lead ---
    for j in range(H):
        abs_list = abs_errors_per_lead[j]
        sq_list  = sq_errors_per_lead[j]

        mae  = float(np.mean(abs_list)) 
        mse  = float(np.mean(sq_list))  
        rmse = float(np.sqrt(mse))      

        rows.append({
            "model": model,
            "time_step": str(j + 1),
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse
        })

    # --- Add weighted aggregate row ---
    df_tmp = pd.DataFrame([r for r in rows if r["model"] == model])

    avg_mae = float(np.average(df_tmp["MAE"]))
    avg_mse = float(np.average(df_tmp["MSE"]))
    avg_rmse = float(np.sqrt(avg_mse))

    rows.append({
        "model": model,
        "time_step": "avg",
        "MAE": avg_mae,
        "MSE": avg_mse,
        "RMSE": avg_rmse
    })

metrics_df = pd.DataFrame(rows)
metrics_table = wandb.Table(data=metrics_df, columns=metrics_df.columns.tolist())
wandb.log({"rolling_stepwise_test_results": metrics_table})

In [ ]:
for model in predictor.model_names():
    predictions = predictor.predict(test_df[:-config["prediction_length"]], model=model)
    fig = predictor.plot(test_df, predictions, max_history_length=config["prediction_length"]*7)

    wandb.log({f"plot_test_{model}": wandb.Image(fig)})

In [ ]:
wandb.finish()

## Testing

In [ ]:
from autogluon.timeseries import TimeSeriesPredictor

model_path = r"C:\Users\PC\Desktop\Tech\VGU\BASWAP\baswap\models\autogluonTS\autogluon_h"

predictor = TimeSeriesPredictor.load(path=model_path)

In [ ]:
for model in predictor.model_names():
    predictions = predictor.predict(test_df[:-config["prediction_length"]], model=model)
    fig = predictor.plot(test_df, predictions, max_history_length=config["prediction_length"]*7)

In [ ]:
import numpy as np
import pandas as pd

H = config["prediction_length"]
input_window = 2 * H

rows = []

model_names = predictor.model_names()

for model in model_names:

    abs_errors_per_lead = [[] for _ in range(H)]
    sq_errors_per_lead  = [[] for _ in range(H)]

    for item in test_df.item_ids:

        series_true = test_df.loc[item][config["target"]]

        t_arr = np.asarray(series_true)
        N = len(t_arr)
        if N < input_window + H:
            continue

        num_rolls = N - (input_window + H) + 1

        for roll in range(num_rolls):

            input_start = 0
            input_end   = roll + input_window
            future_start = input_end
            future_end   = input_end + H

            # --- FIX: AutoGluon REQUIRES a TimeSeriesDataFrame, not a Series ---
            input_slice = test_df.loc[[item]].iloc[input_start:input_end]

            # Predict only this item
            pred_df = predictor.predict(input_slice, model=model)

            # AG output: MultiIndex → (item, timestamp) → 'mean'
            pred_series = pred_df.loc[item]["mean"].values

            true_future = t_arr[future_start:future_end]

            min_len = min(len(true_future), len(pred_series))
            if min_len == 0:
                continue

            for j in range(min_len):
                t = true_future[j]
                p = pred_series[j]
                if np.isnan(t) or np.isnan(p):
                    continue

                err = p - t
                abs_errors_per_lead[j].append(abs(err))
                sq_errors_per_lead[j].append(err ** 2)

    # --- Compute metrics for each lead ---
    for j in range(H):
        abs_list = abs_errors_per_lead[j]
        sq_list  = sq_errors_per_lead[j]

        mae  = float(np.mean(abs_list)) 
        mse  = float(np.mean(sq_list))  
        rmse = float(np.sqrt(mse))      

        rows.append({
            "model": model,
            "lead_time": str(j + 1),
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse
        })

    # --- Add weighted aggregate row ---
    df_tmp = pd.DataFrame([r for r in rows if r["model"] == model and r["lead_time"] != "avg"])

    avg_mae = float(np.average(df_tmp["MAE"]))
    avg_mse = float(np.average(df_tmp["MSE"]))
    avg_rmse = float(np.sqrt(avg_mse))

    rows.append({
        "model": model,
        "lead_time": "avg",
        "MAE": avg_mae,
        "MSE": avg_mse,
        "RMSE": avg_rmse
    })

metrics_df = pd.DataFrame(rows)
metrics_df[metrics_df["model"] == "WeightedEnsemble"]
